# Import thư viện

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Softmax, Dropout
from nltk.corpus import stopwords
import re
from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

ModuleNotFoundError: No module named 'tensorflow'

# Nhập thông tin phim

In [ ]:
# Tên phim
film_title = input('Nhập tên phim')

# Thể loại phim
genre = input('Thể loại phim:')

# Rating phim
rating = input('Rating phim:')


# Tóm tắt phim
synopsis = input('Tóm tắt phim:')

# Tính điểm số sentiment

## Crawl tweet về phim trên twitter dựa vào tên phim

In [ ]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
# options.add_argument('--incognito')
options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path='C:\chromedriver', options=options)
driver.implicitly_wait(0)
actions = ActionChains(driver)

In [ ]:
url = "https://twitter.com/login"
driver.get(url)

In [ ]:
element = WebDriverWait(driver, 600).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete="username"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
soup = BeautifulSoup(html_of_interest, 'lxml')

sleep(1)
driver.find_element(By.CSS_SELECTOR, 'input[autocomplete="username"]').send_keys('cap246k291@gmail.com')
actions.send_keys(Keys.ENTER)
actions.perform()

sleep(5)
try:
    driver.find_element(By.CSS_SELECTOR, 'input[autocapitalize="none"]').send_keys('@cap246k291')
    actions.send_keys(Keys.ENTER)
    actions.perform()
except:
    pass

sleep(5)

driver.find_element(By.CSS_SELECTOR, 'input[autocomplete="current-password"]').send_keys('cap2@2023')
actions.send_keys(Keys.ENTER)
actions.perform()

sleep(5)

In [ ]:
driver.find_element(By.CSS_SELECTOR, 'input[aria-label="Search query"]').send_keys('(#%s) lang:en'%title_film)
actions.send_keys(Keys.ENTER)
actions.perform()

list_tweet = []

while len(list(set(list_tweet))) < 100:
    element = WebDriverWait(driver, 600).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[aria-label="Timeline: Search timeline"]')))
    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
    soup = BeautifulSoup(html_of_interest, 'lxml')

    rv = soup.select('div[data-testid="tweetText"]')
    for i in rv:
        list_tweet.append(i.text)
    
    a = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
    a[-1].location_once_scrolled_into_view

    sleep(3)

In [ ]:
data_tweet = pd.DataFrame(list(set(list_tweet)), columns=['Tweet'])

In [3]:
df_imdb = pd.read_csv('..\Sentiment Reviews Film\Reviews IMDB Dataset.csv')

X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

text_clsf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

# Tính điểm số sentiment
sentiment_score = (text_clsf.predict(data_tweet['Tweet'].values).tolist().count('positive')/len(data_tweet['Tweet']))*100